# Trabajo de Juan

## Problema

El problema consiste en predecir la probabilidad de atraso de los vuelos que aterrizan o despegan del aeropuerto de Santiago
de Chile (SCL). Para eso les entregamos un dataset usando datos públicos y reales donde cada fila corresponde a un vuelo
que aterrizó o despegó de SCL. Para cada vuelo se cuenta con la siguiente información:

* **Fecha-I** : Fecha y hora programada del vuelo.
* **Vlo-I** : Número de vuelo programado.
* **Ori-I** : Código de ciudad de origen programado.
* **Des-I** : Código de ciudad de destino programado.
* **Emp-I** : Código aerolínea de vuelo programado.
* **Fecha-O** : Fecha y hora de operación del vuelo.
* **Vlo-O** : Número de vuelo de operación del vuelo.
* **Ori-O** : Código de ciudad de origen de operación
* **Des-O** : Código de ciudad de destino de operación.
* **Emp-O** : Código aerolínea de vuelo operado.
* **DIA** : Día del mes de operación del vuelo.
* **MES** : Número de mes de operación del vuelo.
* **AÑO** : Año de operación del vuelo.
* **DIANOM** : Día de la semana de operación del vuelo.
* **TIPOVUELO** : Tipo de vuelo, I =Internacional, N =Nacional.
* **OPERA** : Nombre de aerolínea que opera.
* **SIGLAORI** : Nombre ciudad origen.
* **SIGLADES** : Nombre ciudad destino.

## Desarrollo

### 0. Obtención de datos

In [ ]:
# Se importan las librerías necesarias para el problema
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import missingno as msng

# import warnings
# warnings.filterwarnings('ignore')
%matplotlib inline
plt.rcParams["figure.figsize"] = (15, 10)

In [ ]:
df = pd.read_csv("../data/dataset_SCL.csv")

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.describe()

### 1. ¿Cómo se distribuyen los datos? ¿Qué te llama la atención o cuál es tu conclusión sobre esto?

In [ ]:
vuelos_aerolineas = df["OPERA"].value_counts()
plt.figure(figsize=(10, 6))
sns.set(style="darkgrid")
sns.barplot(x=vuelos_aerolineas.index, y=vuelos_aerolineas.values, alpha=0.9)
plt.title("Cantidad de Vuelos por Aerlínea")
plt.ylabel("Número de Vuelos", fontsize=12)
plt.xlabel("Aerlínea", fontsize=12)
plt.xticks(rotation=90)
plt.show()

In [ ]:
vuelos_dia = df["DIA"].value_counts()
plt.figure(figsize=(10, 6))
sns.set(style="darkgrid")
sns.barplot(x=vuelos_dia.index, y=vuelos_dia.values, color="lightblue", alpha=0.8)
plt.title("Cantidad de Vuelos por día del Mes")
plt.ylabel("Número de Vuelos", fontsize=12)
plt.xlabel("Día del Mes", fontsize=12)
plt.xticks(rotation=90)
plt.show()

In [ ]:
vuelos_mes = df["MES"].value_counts()
sns.set(style="darkgrid")
sns.barplot(x=vuelos_mes.index, y=vuelos_mes.values, color="lightblue", alpha=0.8)
plt.title("Cantidad de Vuelos por día del Mes")
plt.ylabel("Número de Vuelos", fontsize=12)
plt.xlabel("Día del Mes", fontsize=12)
plt.xticks(rotation=90)
plt.show()

In [ ]:
vuelos = df["DIANOM"].value_counts()
dias = [
    vuelos.index[2],
    vuelos.index[5],
    vuelos.index[4],
    vuelos.index[1],
    vuelos.index[0],
    vuelos.index[6],
    vuelos.index[3],
]
valores_dias = [
    vuelos.values[2],
    vuelos.values[5],
    vuelos.values[4],
    vuelos.values[1],
    vuelos.values[0],
    vuelos.values[6],
    vuelos.values[3],
]

sns.set(style="darkgrid")
sns.barplot(x=dias, y=valores_dias, color="lightblue", alpha=0.8)
plt.title("Cantidad de Vuelos por Día de la Semana")
plt.ylabel("Número de Vuelos", fontsize=12)
plt.xlabel("Día de la Semana", fontsize=12)
plt.xticks(rotation=90)
plt.show()

In [ ]:
vuelos = df["TIPOVUELO"].value_counts()
sns.set(style="darkgrid")
sns.barplot(x=vuelos.index, y=vuelos.values, alpha=0.9)
plt.title("Cantidad de Vuelos por Tipo de Vuelo")
plt.ylabel("Número de Vuelos", fontsize=12)
plt.xlabel("Tipo de Vuelo", fontsize=12)
plt.show()

In [ ]:
vuelos = df["SIGLADES"].value_counts()
sns.set(style="darkgrid")
sns.barplot(x=vuelos.index, y=vuelos.values, color="lightblue", alpha=0.8)
plt.title("Cantidad de Vuelos por Destino")
plt.ylabel("Número de Vuelos", fontsize=12)
plt.xlabel("Destino", fontsize=12)
plt.xticks(rotation=90)

plt.show()

Respecto a las distribuciones, se puede ver que dentro de todo la data está distribuida de manera bastante balanceada. De todas formas se pueden notar ciertas particularidades en algunas columnas: 
* En las aerolíneas LATAM Airlines es la con mayor cantidad de vuelo debido a que es la más grande aquí en chile y es la que tiene mayor cantidad de destinos. SKY Airlines sería la segunda más grande y de ahí en adelante las demás son bastante similares en cuanto a vuelos. 
* En cuanto a los días del mes, se puede ver como el día 31 tiene menor cantidad que los demás días y esto se puede deber a que hay menos días 31 durante el año.
* Con los días de semana, el día sábado es el único en el que se nota una diferencia. Aquí puede ser porque la gente prefiere ese día para quedarse en donde sea que esté y aprovecharlo en caso que esté de viaje para poder recorrer o solamente descansar.
* Por parte de los destinos se puede ver que gran parte se lo llevan las ciudades chilenas del norte. Esto puede ser por un alto flujo en minería.

### 2. Genera las columnas adicionales y luego expórtelas en un archivo synthetic_features.csv :

* **temporada_alta** : 1 si **Fecha-I** está entre 15-Dic y 3-Mar, o 15-Jul y 31-Jul, o 11-Sep y 30-Sep, 0 si no

In [ ]:
from datetime import datetime


def temporada_alta(fecha):
    fecha_año = int(fecha.split("-")[0])
    fecha = datetime.strptime(fecha, "%Y-%m-%d %H:%M:%S")
    range1_min = datetime.strptime("15-Dec", "%d-%b").replace(year=fecha_año)
    range1_max = datetime.strptime("31-Dec", "%d-%b").replace(year=fecha_año)
    range2_min = datetime.strptime("1-Jan", "%d-%b").replace(year=fecha_año)
    range2_max = datetime.strptime("3-Mar", "%d-%b").replace(year=fecha_año)
    range3_min = datetime.strptime("15-Jul", "%d-%b").replace(year=fecha_año)
    range3_max = datetime.strptime("31-Jul", "%d-%b").replace(year=fecha_año)
    range4_min = datetime.strptime("11-Sep", "%d-%b").replace(year=fecha_año)
    range4_max = datetime.strptime("30-Sep", "%d-%b").replace(year=fecha_año)

    if (
        (fecha >= range1_min and fecha <= range1_max)
        or (fecha >= range2_min and fecha <= range2_max)
        or (fecha >= range3_min and fecha <= range3_max)
        or (fecha >= range4_min and fecha <= range4_max)
    ):
        return 1
    else:
        return 0

In [ ]:
df["temporada_alta"] = df["Fecha-I"].apply(temporada_alta)

In [ ]:
df["temporada_alta"].value_counts()

* **dif_min** : diferencia en minutos entre **Fecha-O** y **Fecha-I** .

In [ ]:
def dif_min(data):
    fecha_o = datetime.strptime(data["Fecha-O"], "%Y-%m-%d %H:%M:%S")
    fecha_i = datetime.strptime(data["Fecha-I"], "%Y-%m-%d %H:%M:%S")
    dif_min = ((fecha_o - fecha_i).total_seconds()) / 60
    return dif_min

In [ ]:
df["dif_min"] = df.apply(dif_min, axis=1)

* **atraso_15** : 1 si **dif_min** > 15, 0 si no.

In [ ]:
df["atraso_15"] = np.where(df["dif_min"] > 15, 1, 0)

In [ ]:
df["atraso_15"].value_counts()

* **periodo_dia** : mañana (entre 5:00 y 11:59), tarde (entre 12:00 y 18:59) y noche (entre 19:00 y 4:59), en base a **Fecha-I**.

In [ ]:
def get_periodo_dia(fecha):
    fecha_time = datetime.strptime(fecha, "%Y-%m-%d %H:%M:%S").time()
    mañana_min = datetime.strptime("05:00", "%H:%M").time()
    mañana_max = datetime.strptime("11:59", "%H:%M").time()
    tarde_min = datetime.strptime("12:00", "%H:%M").time()
    tarde_max = datetime.strptime("18:59", "%H:%M").time()
    noche_min1 = datetime.strptime("19:00", "%H:%M").time()
    noche_max1 = datetime.strptime("23:59", "%H:%M").time()
    noche_min2 = datetime.strptime("00:00", "%H:%M").time()
    noche_max2 = datetime.strptime("4:59", "%H:%M").time()

    if fecha_time > mañana_min and fecha_time < mañana_max:
        return "mañana"
    elif fecha_time > tarde_min and fecha_time < tarde_max:
        return "tarde"
    elif (fecha_time > noche_min1 and fecha_time < noche_max1) or (
        fecha_time > noche_min2 and fecha_time < noche_max2
    ):
        return "noche"
    return "unknown"

In [ ]:
df["periodo_dia"] = df["Fecha-I"].apply(get_periodo_dia)

In [ ]:
df[["temporada_alta", "dif_min", "atraso_15", "periodo_dia"]].to_csv(
    "../data/synthetic_features.csv", index=False
)

### 3. ¿Cómo se compone la tasa de atraso por destino, aerolínea, mes del año, día de la semana, temporada, tipo de vuelo? ¿Qué variables esperarías que más influyeran en predecir atrasos?

* ¿Cómo se compone la tasa de atraso por destino, aerolínea, mes del año, día de la semana, temporada, tipo de vuelo?

In [ ]:
def calcular_tasa(df, columna):

    dic_atrasos = {}
    for _, row in df.iterrows():
        if row["atraso_15"] == 1:
            if row[columna] not in dic_atrasos:
                dic_atrasos[row[columna]] = 1
            else:
                dic_atrasos[row[columna]] += 1

    total_values = df[columna].value_counts()

    dic_tasas = {}
    for name, total in total_values.iteritems():
        if name in dic_atrasos:
            dic_tasas[name] = round(total / dic_atrasos[name], 2)
        else:
            dic_tasas[name] = 0

    return pd.DataFrame.from_dict(data=dic_tasas, orient="index", columns=["Tasa (%)"])

In [ ]:
tasas_destinos = calcular_tasa(df, "SIGLADES")
tasas_destinos

In [ ]:
tasas_destinos_values = df["SIGLADES"].value_counts().index
plt.figure(figsize=(20, 5))
sns.set(style="darkgrid")
sns.barplot(
    x=df["SIGLADES"].value_counts().index, y=tasas_destinos["Tasa (%)"], alpha=0.75
)
plt.title("Tasa de Retraso por Destino")
plt.ylabel("Tasa de Retraso [%]", fontsize=12)
plt.xlabel("Destino", fontsize=12)
plt.xticks(rotation=90)
plt.show()

In [ ]:
tasas_aerolineas = calcular_tasa(df, "OPERA")
tasas_aerolineas

In [ ]:
tasas_aerolineas_values = df["OPERA"].value_counts().index

plt.figure(figsize=(10, 5))
sns.set(style="darkgrid")
sns.barplot(x=tasas_aerolineas_values, y=tasas_aerolineas["Tasa (%)"], alpha=0.75)
plt.title("Tasa de Retraso por Aerlínea")
plt.ylabel("Tasa de Retraso [%]", fontsize=12)
plt.xlabel("Aerolínea", fontsize=12)
plt.xticks(rotation=90)
plt.show()

In [ ]:
tasas_meses = calcular_tasa(df, "MES")
tasas_meses

In [ ]:
tasas_meses_values = df["MES"].value_counts().index

plt.figure(figsize=(10, 5))
sns.set(style="darkgrid")
sns.barplot(x=tasas_meses_values, y=tasas_meses["Tasa (%)"], color="blue", alpha=0.75)
plt.title("Tasa de Retraso por Mes")
plt.ylabel("Tasa de Retraso [%]", fontsize=12)
plt.xlabel("Meses", fontsize=12)
plt.xticks(rotation=90)
plt.ylim(0, 10)
plt.show()

In [ ]:
tasas_dias = calcular_tasa(df, "DIANOM")
tasas_dias

In [ ]:
tasas_dias_values = df["DIANOM"].value_counts().index

sns.set(style="darkgrid")
plt.figure(figsize=(8, 6))
sns.barplot(x=tasas_dias_values, y=tasas_dias["Tasa (%)"], color="blue", alpha=0.75)
plt.title("Tasa de Retraso por Día")
plt.ylabel("Tasa de Retraso [%]", fontsize=12)
plt.xlabel("Días", fontsize=12)
plt.xticks(rotation=90)
plt.ylim(0, 7)
plt.show()

In [ ]:
tasas_temporada = calcular_tasa(df, "temporada_alta")
tasas_temporada

In [ ]:
tasas_temporada_values = df["temporada_alta"].value_counts().index

plt.figure(figsize=(8, 5))
sns.set(style="darkgrid")
sns.barplot(x=tasas_temporada_values, y=tasas_temporada["Tasa (%)"])
plt.title("Tasa de Retraso por Temporada")
plt.ylabel("Tasa de Retraso [%]", fontsize=12)
plt.xlabel("Temporada", fontsize=12)
plt.xticks(rotation=90)
plt.ylim(0, 6)
plt.show()

In [ ]:
tasas_tipovuelo = calcular_tasa(df, "TIPOVUELO")
tasas_tipovuelo

In [ ]:
tasas_tipovuelo_values = df["TIPOVUELO"].value_counts().index

plt.figure(figsize=(5, 5))
sns.set(style="darkgrid")
sns.barplot(x=tasas_tipovuelo_values, y=tasas_tipovuelo["Tasa (%)"])
plt.title("Tasa de Retraso por Tipo de Vuelo")
plt.ylabel("Tasa de Retraso [%]", fontsize=12)
plt.xlabel("Tipo de Vuelo", fontsize=12)
plt.ylim(0, 7)
plt.show()

In [ ]:
tasa_periododia = calcular_tasa(df, "periodo_dia")
tasa_periododia

In [ ]:
tasas_periodo_values = df["periodo_dia"].value_counts().index

plt.figure(figsize=(5, 5))
sns.set(style="darkgrid")
sns.barplot(x=tasas_periodo_values, y=tasa_periododia["Tasa (%)"])
plt.title("Tasa de Retraso por Periodo")
plt.ylabel("Tasa de Retraso [%]", fontsize=12)
plt.xlabel("Tipo de Periodo", fontsize=12)
plt.ylim(3, 7)
plt.show()

* ¿Qué variables esperarías que más influyeran en predecir atrasos?

Me parece que las que más podrían influir serían:
* **Aerolíneas**: Dependiendo de la calidad de gestión de cada aerolínea puede ser que afecte en sus programaciones. También puede que afecte el tipo de aerolínea, si es una low cost por ejemplo, estas llevan menos equipaje ya que es un costo extra para cada pasajero por lo que el tiempo de carga es menor, corriendo menor riesgo de retraso. 
* **Tipo de Vuelo**: esto puede afectar en cómo esté el clima del país de destino. Por ejemplo si se tiene que volar a argentina y se tiene que cruzar por la cordillera y hay un mal clima, es posible que se tenga que esperar un rato para poder salir.
* **Mes**: Esto va un poco de la mano con lo anterior, si es que es un mes del año donde el clima no es lo suficientemente adecuado, se tendría que esperar.  
 

### 4. Entrena uno o varios modelos (usando el/los algoritmo(s) que prefieras) para estimar la probabilidad de atraso de un vuelo. Siéntete libre de generar variables adicionales y/o complementar con variables externas.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
data = df[["OPERA", "MES", "TIPOVUELO", "SIGLADES", "DIANOM", "atraso_15"]]

In [ ]:
features = pd.concat(
    [
        pd.get_dummies(data["OPERA"], prefix="OPERA"),
        pd.get_dummies(data["TIPOVUELO"], prefix="TIPOVUELO"),
        pd.get_dummies(data["MES"], prefix="MES"),
    ],
    axis=1,
)
label = data["atraso_15"]

In [ ]:
x_train_reg, x_test_reg, y_train_reg, y_test_reg = train_test_split(
    features, label, test_size=0.33, random_state=42, shuffle=True, stratify=label
)

In [ ]:
x_train_reg.shape, x_test_reg.shape

In [ ]:
y_train_reg.value_counts("%")

In [ ]:
y_test_reg.value_counts("%")

#### Utilizando Regresión logística

In [ ]:
logReg = LogisticRegression()
modelreg = logReg.fit(x_train_reg, y_train_reg)

In [ ]:
y_pred = modelreg.predict(x_test_reg)

#### Métricas Regresión Logística

In [ ]:
confusion_matrix(y_test_reg, y_pred)

In [ ]:
print(classification_report(y_test_reg, y_pred))

#### Utilizando XGBoost

In [ ]:
import xgboost as xgb
from xgboost import plot_importance

In [ ]:
modelxgb1 = xgb.XGBClassifier(random_state=1, learning_rate=0.01)
modelxgb1 = modelxgb1.fit(x_train_reg, y_train_reg)

In [ ]:
y_predxgb = modelxgb1.predict(x_test_reg)

#### Métricas XGBoost

In [ ]:
confusion_matrix(y_test_reg, y_predxgb)

In [ ]:
print(classification_report(y_test_reg, y_predxgb))

#### Utilizando XGBoost dejando Features más importantes 

In [ ]:
modelxgb1.feature_importances_

In [ ]:
plt.figure(figsize=(13, 10))
plot_importance(modelxgb1)

In [ ]:
features_importantes = features[
    [
        "MES_7",
        "TIPOVUELO_I",
        "OPERA_Copa Air",
        "OPERA_Latin American Wings",
        "MES_12",
        "OPERA_Grupo LATAM",
        "MES_10",
        "OPERA_JetSmart SPA",
        "OPERA_Air Canada",
        "MES_9",
        "OPERA_American Airlines",
    ]
]
labels_ = label

In [ ]:
x_train2, x_test2, y_train2, y_test2 = train_test_split(
    features_importantes, labels_, test_size=0.33, random_state=42
)

In [ ]:
modelxgb = xgb.XGBClassifier(
    random_state=1, learning_rate=0.01, subsample=1, max_depth=10
)
modelxgb = modelxgb.fit(x_train2, y_train2)

In [ ]:
y_predxgb = modelxgb.predict(x_test2)

#### Métricas XGBoost dejando Features más importantes 

In [ ]:
confusion_matrix(y_test2, y_predxgb)

In [ ]:
print(classification_report(y_test2, y_predxgb))

### 5. Evalúa tu modelo. ¿Qué performance tiene? ¿Qué métricas usaste para evaluar esa performance y por qué? ¿Por qué elegiste ese algoritmo en particular? ¿Qué variables son las que más influyen en la predicción? ¿Cómo podrías mejorar laperformance?

Los modelos utilizados en la sección anterior fueron:
* **Regresión Logística**: Fue seleccionado ya que es el modelo más simple para poder resolver problemas de clasificación.
* **XGBoost**: Fue seleccionado porque en general tiene un gran desempeño y entrega buenos resultados ya sea para regresión y clasificación, además es bastante rápido y facilita el tuneo de hiperparámetros.

Los resultados obtenidos con ambos modelos fueron muy malos. Se utilizó matriz de confusión para validar los resultados. La cantidad de falsos negativos fue muy alta. Una de las posibles razones, la más clara, es el desbalanceo que existe en la clase 'atraso_15'. Del total de datos un 82% corresponde a no atraso y el 18% restante a atraso.
Para poder mejorar la performance se tomaron dos decisiones, la primera aplicar un Grid Search CV sobre XGBoost para tunear de mejor manera los hiperparámetros y la segunda realizar un upsampling de la clase desbalanceada, en este caso los atrasos.



### Grid Search CV

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
parameters = {
    "learning_rate": [0.01, 0.05, 0.1],
    "n_estimators": [50, 100, 150],
    "subsample": [0.5, 0.9],
}

# modelxgb_GridCV = GridSearchCV(modelxgb, param_grid = parameters,
#    cv = 2, n_jobs=-1, verbose=1).fit(x_train_reg, y_train_reg)

In [ ]:
# y_predxgb_grid = modelxgb_GridCV.predict(x_test_reg)

In [ ]:
# confusion_matrix(y_test_reg, y_predxgb_grid)

### Upsampling

In [ ]:
data["atraso_15"].value_counts()

In [ ]:
# Resample
from sklearn.utils import resample

data_no_retraso = data[data["atraso_15"] == 0]
data_atraso = data[data["atraso_15"] == 1]

data_atraso_upsampled = resample(
    data_atraso,
    replace=True,  # sample with replacement
    n_samples=30000,  # to match majority class
    random_state=42,
)  # reproducible results

data_upsampled = pd.concat([data_no_retraso, data_atraso_upsampled])

In [ ]:
data_upsampled["atraso_15"].value_counts()

In [ ]:
features_upsampled = pd.concat(
    [
        pd.get_dummies(data_upsampled["OPERA"], prefix="OPERA"),
        pd.get_dummies(data_upsampled["TIPOVUELO"], prefix="TIPOVUELO"),
        pd.get_dummies(data_upsampled["MES"], prefix="MES"),
    ],
    axis=1,
)
label_upsampled = data_upsampled["atraso_15"]

In [ ]:
(
    x_upsampled_train,
    x_upsampled_test,
    y_upsampled_train,
    y_upsampled_test,
) = train_test_split(
    features_upsampled, label_upsampled, test_size=0.33, random_state=42
)

In [ ]:
modelxgb = xgb.XGBClassifier(
    random_state=1, learning_rate=0.01, subsample=1, max_depth=10
)
modelxgb.fit(x_upsampled_train, y_upsampled_train)

In [ ]:
y_upsampled_predxgb = modelxgb.predict(x_upsampled_test)

In [ ]:
confusion_matrix(y_upsampled_test, y_upsampled_predxgb)

In [ ]:
modelxgb.score(x_upsampled_test, y_upsampled_test)

In [ ]:
print(classification_report(y_upsampled_test, y_upsampled_predxgb))

Observando los dos métodos para incrementar la performance, se puede ver como el modelo no mejoró. Esto puede ser ya que la data creada al upsamplear no tiene ningún sentido. Si se pudiese pudiese balancear la clase tal vez los modelos tendrían un mejor desempeño.

# Trabajo de Francisco

Disclaimer: Tuve que adaptar algunas cosas en el código de Juan, se caía en algunas partes. Dejé comentadas las líneas que modifiqué.

Si bien se me pidió escoger un modelo de Juan (e implementar mejoras sobre el **modelo**), quiero mostrar que con un entendimiento un poco más adecuado de los datos se puede mejorar la performance con un enfoque sencillo y muy directo (usando el mismo modelo).

Voy a eliminar las siguientes columnas:

    - dif_min: Pertinente a la label, no puedo utilizarla como variable predictora
    - periodo_dia: Voy a dejar la hora como atributo, resumirla en 3 categorías macro no me parece beneficioso. Creo que tenemos la suficiente cantidad de datos como para no tener que simplificar el atributo, se pierde información haciéndolo.
    - Vlo-I: Esta variable no afectó la performance, decidí eliminarla.
    - AÑO: 2017 siempre, no aporta información discriminativa.
    - (*)-O: Todas las columnas que son pertinentes a la operación. Voy a asumir que la información de la operación corresponde a cuando el vuelo **ya sucedio**. No va a estar disponible en los datos que queremos predecir, no puedo ocuparlas como variables predictoras
    
Me quedo con la label que planteó Juan. Puede cuestionarse (es una decisión importante), pero ocuparé la misma para demostrar la mejora en la performance. Además me parece sensata.



In [ ]:
from sklearn.preprocessing import OneHotEncoder

features = df.copy()
# elimino las columnas
features = features.drop(
    [
        "dif_min",
        "periodo_dia",
        "Vlo-I",
        "AÑO",
        "Vlo-O",
        "Ori-O",
        "Des-O",
        "Emp-O",
        "OPERA",
        "SIGLAORI",
        "SIGLADES",
        "temporada_alta",
    ],
    axis=1,
)
label = features.pop("atraso_15")

# convierto las columnas fecha a datetime
features["Fecha-I"] = pd.to_datetime(features["Fecha-I"])
features["Fecha-O"] = pd.to_datetime(features["Fecha-O"])
features["DIA-I"] = features["Fecha-I"].dt.day.values
features.head()

Las columnas Mes,Dia,DIANOM estan construidas a partir de *Fecha-O*, voy a construir las mismas variables a partir de *Fecha-I* y ocupar esas. Voy a corregir *DIANOM* cuando haya una diferencia entre *Fecha-O* y *Fecha-I*. Este paso es probablemente despreciable (son pocos los datos en que cambia el dia), pero me pareció correcto incluirlo.

In [ ]:
display(
    features[features["DIA-I"] != features["DIA"]]
)  # en estos casos tengo que corregir el dia de la semana, que me interesa.

features["diff_days"] = features["Fecha-O"].dt.to_period("D").astype(int) - features[
    "Fecha-I"
].dt.to_period("D").astype(
    int
)  # .dt.days
lista_dias = ["Lunes", "Martes", "Miercoles", "Jueves", "Viernes", "Sabado", "Domingo"]


def fix_day(row):
    nombre_dia_op = row["DIANOM"]
    resta = row["diff_days"]
    index_op = lista_dias.index(nombre_dia_op)
    index_nuevo = (index_op - resta) % 7  # para rotar en la lista lunes->domingo
    return lista_dias[index_nuevo]


features.loc[features["DIA-I"] != features["DIA"], "DIANOM"] = features[
    features["DIA-I"] != features["DIA"]
].apply(fix_day, axis=1)
features[features["diff_days"] > 0]

In [ ]:
features["MES-I"] = features["Fecha-I"].dt.month
features["Hora"] = (
    features["Fecha-I"].dt.hour + features["Fecha-I"].dt.minute / 60
)  # tratamos la hora como float
features = features.drop(["Fecha-I", "Fecha-O", "DIA", "MES", "diff_days"], axis=1)
features.head()

In [ ]:
cols = [
    columna
    for cont, columna in enumerate(features.columns)
    if features.dtypes[cont] != "float64"
]
features[cols] = features[cols].astype("category")
features.dtypes

In [ ]:
My_x_train, My_x_test, My_y_train, My_y_test = train_test_split(
    features, label, test_size=0.33, random_state=42, shuffle=True, stratify=label
)
My_x_train.shape, My_x_test.shape, My_x_train.columns

In [ ]:
from sklearn.compose import make_column_selector as selector
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown="ignore")

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, selector(dtype_include="float64")),
        ("cat", categorical_transformer, selector(dtype_include="category")),
    ]
)
My_x_train, My_x_test = preprocessor.fit_transform(My_x_train), preprocessor.transform(
    My_x_test
)

In [ ]:
My_modelxgb = xgb.XGBClassifier(random_state=1)
My_modelxgb = My_modelxgb.fit(My_x_train, My_y_train)
My_y_predxgb = My_modelxgb.predict(My_x_test)
print(classification_report(My_y_test, My_y_predxgb))
confusion_matrix(My_y_test, My_y_predxgb)

Hay una mejora significativa en f1-score para la clase positiva (minoría), usando el mismo modelo sin ninguna sincronizacion de parámetros.

# Evaluación sugerida

Creo que accuracy y f1-score no son suficientes como métricas. El desafío es determinar la probabilidad de atraso, estas métricas estan midiendo la cantidad de aciertos considerando un treshold probabilístico *arbitrario*: 0.5. Sugeriría evaluar con Roc Curve y Auc Score, que se abstraen del treshold y miden directamente sobre las probabilidades.

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score

My_y_proba = My_modelxgb.predict_proba(My_x_test)
y_proba_reg = modelreg.predict_proba(x_test_reg)
y_proba_xgb = modelxgb1.predict_proba(x_test_reg)

# roc curve for models
fpr1, tpr1, thresh1 = roc_curve(My_y_test, My_y_proba[:, 1], pos_label=1)
fpr_reg, tpr_reg, thresh_reg = roc_curve(y_test_reg, y_proba_reg[:, 1], pos_label=1)
fpr_xgb, tpr_xgb, thresh_xgb = roc_curve(y_test_reg, y_proba_xgb[:, 1], pos_label=1)
# auc scores
auc_score1 = roc_auc_score(My_y_test, My_y_proba[:, 1])
auc_score_reg = roc_auc_score(y_test_reg, y_proba_reg[:, 1])
auc_score_xgb = roc_auc_score(y_test_reg, y_proba_xgb[:, 1])

print("My Method:", auc_score1)
print("Juan Logistic Regression:", auc_score_reg)
print("Juan Xgboost:", auc_score_xgb)

plt.plot(fpr1, tpr1, linestyle="--", color="orange", label="My Method")
plt.plot(
    fpr_reg,
    tpr_reg,
    linestyle="--",
    color="blue",
    label="Juan Logistic Regression",
    alpha=0.5,
)
plt.plot(
    fpr_xgb, tpr_xgb, linestyle="--", color="black", label="Juan Xgboost", alpha=0.5
)

# title
plt.title("ROC curve")
# x label
plt.xlabel("False Positive Rate")
# y label
plt.ylabel("True Positive rate")

plt.legend(loc="best")
plt.show();

### Se nota una mejora en la performance.

## Para seguir mejorando el enfoque
Hay varias cosas que hacer, lo más directo es sincronizar parámetros o probar distintos modelos (lightgbm suele darme buenos resultados). En mi experiencia, sin embargo, esto suele traer mejoras marginales.

Creo que hay 2 cosas que podría traer mejoras importantes al sistema:

    - Label: El método que estamos planteando para escoger la label. Aquí estamos perdiendo información sobre vuelos con atrasos más grandes que otros. Esta información puede ser importante y sería interesante incluirla. Si es que no lograra determinar por mi mismo cual es la mejor manera, consideraría hablar con el cliente sobre que es lo que ellos consideran atraso (nosotros (Juan) lo definimos como > 15min)
    
    - Enriquecimiento: Sinceramente no sé nada del rubro de las aerolíneas. Sospecho que uno de los factores más influyentes en los atrasos son las variables meteorológicas. Haría un enriquecimiento con estas variables de acuerdo a la ruta (orig-dest) y la (Fecha-I), ¿qué clima está pronosticado?
    
    - Time Series: ¿Son los datos independientes? después de todo, esto es en esencia una Serie de Tiempo, quizás la presencia de atrasos en cierto momento puede ayudar a determinar si van a haber atrasos en un futuro cercano. (Vuelos afectados por el mismo fenómeno). Esto me faltó por explorar.
        - Si interpretamos esto como una Serie de Tiempo, habría que cambiar la metodología. El conjunto de test debiese ser el último período de la Serie y debiésemos poder predecir usando los datos históricos.


In [ ]:
# guardamos el modelo para ser usado por la api
import joblib

My_modelxgb.save_model("../objects/model.json")
joblib.dump(preprocessor, "../objects/preprocessor.joblib")